In [1]:
import numpy as np
import scipy as sp
from skimage import io
from matplotlib import pyplot as plt
from skimage import transform
from tqdm import tqdm
from time import time
import pickle
import os
import sys
import pandas as pd
from glob import glob
import mrcfile
from skimage import registration as im_reg_met
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

from utils import shift, iou, polar_trfm
from registration import (
    set_integration_intervals,
    laguerre_zeros_precompute, 
    image_fbt_precompute, 
    fbm_registration,
    laguerre_functions_precompute
)
sys.path.append('./compute results/')
from registration_functions import precompute_w_params, run_fbm, run_fbm_laguerre, run_fast_fbm_laguerre

In [2]:
from matrix_utils import *
import cv2

def normalize(im):
    return (im - im.min()) / (im.max() - im.min())

def shift(im, vec):
    mat_trans = get_translation_mat(*vec)
    return cv2.warpAffine(im, get_mat_2x3(mat_trans), 
                          (im.shape[1], im.shape[0]))
    
    
def rotate(im, angle, center=None):
    h, w = im.shape[:2]
    if center is None:
        center = np.array([w // 2, h // 2])
    mat_trans_minus_center = get_translation_mat(-center[0], -center[1])
    mat_rot = get_rotation_mat(angle, radians=False)
    mat_trans_center = get_translation_mat(*center)
    return cv2.warpAffine(im, get_mat_2x3(mat_trans_center @ mat_rot @ mat_trans_minus_center),
                          (im.shape[1], im.shape[0]))


def apply_transform(image, transform_dict, center = None):
    h, w = image.shape[:2]
    if center is None:
        center = np.array([w // 2, h // 2])
        
    ksi = transform_dict['ksi']
    etta_prime = transform_dict['etta']
    omegga_prime = transform_dict['omegga']
    etta = etta_prime - ksi
    omegga = omegga_prime - etta_prime
    eps = transform_dict['eps']
    com_offset = transform_dict['com_offset']

    rho = (2* com_offset**2 * (1 + np.cos(etta + eps))) ** 0.5
    tx, ty = rho * np.cos(ksi), rho * np.sin(ksi)
    alpha = etta + eps + omegga + ksi
#     print('alpha', np.degrees(alpha))

    im_reg = rotate(image, -np.degrees(alpha))

    im_reg = shift(im_reg, (-tx, -ty))
    
    return im_reg, [np.degrees(alpha), tx, ty]


def save_arr_mrc(fname, arr):
    with mrcfile.new(fname, overwrite=True) as mrc:
        mrc.set_data(np.array(arr))

        
def mean_std_normalize(im):
    im = (im - im.mean())
    return im / im.std()


In [3]:
def run_fast_fbm_laguerre(seq, func_parameters):
    df = pd.DataFrame(columns=['x', 'y', 'ang', 'ksi', 'eta_prime', 'omega_prime', 'dft_x', 'dft_y'])
    fbm_seq = [seq[0].copy()]
    fbm_seq_shift = [seq[0].copy()]
    
    fixed_image = normalize(sp.ndimage.gaussian_filter(seq[0].copy(), 1.3))
    func_parameters = fixed_image_precompute(fixed_image,
                                             func_parameters)
    
    for i in tqdm(range(1, len(seq))):
        reg1 = fbm_registration(fixed_image,
                                normalize(sp.ndimage.gaussian_filter(seq[i].copy(), 1.3)),
                                image_radius=image_radius, p_s=pixel_sampling, com_offset=com_offset_initial,
                                method='fast_fbm_laguerre', lag_func_num=lag_func_num, lag_scale=lag_scale,
                                masks=None, shift_by_mask=False, additional_params=func_parameters)
        
        im_reg1, params = apply_transform(seq[i].copy(), reg1, center)
        fbm_seq.append(im_reg1.copy())

#         print('IoU:', iou(seq[0], im_reg1))
        shifts = im_reg_met.phase_cross_correlation(normalize(sp.ndimage.gaussian_filter(seq[0].copy(), 1.3)),
                                                    normalize(sp.ndimage.gaussian_filter(im_reg1.copy(), 1.3)),
                                                    upsample_factor=100)[0][::-1]

        fbm_seq_shift.append(shift(im_reg1, -shifts))
        founded_values = [params[1], params[2], params[0], reg1['ksi'],
                          reg1['etta'], reg1['omegga'], shifts[0], shifts[1]]
        df.loc[i-1] = founded_values
        
    fbm_seq = np.array(fbm_seq)
    return df, fbm_seq, fbm_seq_shift


def run_fbm(seq, func_parameters):
    df = pd.DataFrame(columns=['x', 'y', 'ang', 'ksi', 'eta_prime', 'omega_prime', 'dft_x', 'dft_y'])
    fbm_seq = [seq[0].copy()]
    fbm_seq_shift = [seq[0].copy()]
    fixed_image = normalize(sp.ndimage.gaussian_filter(seq[0].copy(), 1.3))
    func_parameters = fixed_image_precompute(fixed_image,
                                             func_parameters)
    for i in tqdm(range(1, len(seq))):
        reg1 = fbm_registration(fixed_image,
                                normalize(sp.ndimage.gaussian_filter(seq[i].copy(), 1.3)),
                                image_radius=image_radius, p_s=pixel_sampling, com_offset=com_offset_initial,
                                method='fbm', masks=None, shift_by_mask=False,
                                additional_params=func_parameters)
    #                             method='fbm_laguerre', lag_func_num=lag_func_num, lag_scale=lag_scale,
    #                             masks=None, shift_by_mask=False, additional_params=params)
        im_reg1, params = apply_transform(seq[i].copy(), reg1, center)
        fbm_seq.append(im_reg1.copy())

#         print('IoU:', iou(seq[0], im_reg1))
        shifts = im_reg_met.phase_cross_correlation(normalize(sp.ndimage.gaussian_filter(seq[0].copy(), 1.3)),
                                                    normalize(sp.ndimage.gaussian_filter(im_reg1.copy(), 1.3)),
                                                    upsample_factor=100)[0][::-1]

        fbm_seq_shift.append(shift(im_reg1, -shifts))
        founded_values = [params[1], params[2], params[0], reg1['ksi'],
                          reg1['etta'], reg1['omegga'], shifts[0], shifts[1]]
        df.loc[i-1] = founded_values
        
    fbm_seq = np.array(fbm_seq)
    return df, fbm_seq, fbm_seq_shift


def run_fbm_laguerre(seq, func_parameters):
    df = pd.DataFrame(columns=['x', 'y', 'ang', 'ksi', 'eta_prime', 'omega_prime', 'dft_x', 'dft_y'])
    fbm_seq = [seq[0].copy()]
    fbm_seq_shift = [seq[0].copy()]
    fixed_image = normalize(sp.ndimage.gaussian_filter(seq[0].copy(), 1.3))
    func_parameters = fixed_image_precompute(fixed_image,
                                             func_parameters)
    for i in tqdm(range(1, len(seq))):
        reg1 = fbm_registration(fixed_image,
                                normalize(sp.ndimage.gaussian_filter(seq[i].copy(), 1.3)),
                                image_radius=image_radius, p_s=pixel_sampling, com_offset=com_offset_initial,
                                method='fbm_laguerre', masks=None, shift_by_mask=False,
                                lag_func_num=lag_func_num, lag_scale=lag_scale, 
                                additional_params=func_parameters)
        im_reg1, params = apply_transform(seq[i].copy(), reg1, center)
        fbm_seq.append(im_reg1.copy())

#         print('IoU:', iou(seq[0], im_reg1))
        shifts = im_reg_met.phase_cross_correlation(fixed_image,
                                                    normalize(sp.ndimage.gaussian_filter(im_reg1.copy(), 1.3)),
                                                    upsample_factor=100)[0][::-1]

        fbm_seq_shift.append(shift(im_reg1, -shifts))
        founded_values = [params[1], params[2], params[0], reg1['ksi'],
                          reg1['etta'], reg1['omegga'], shifts[0], shifts[1]]
        df.loc[i-1] = founded_values
        
    fbm_seq = np.array(fbm_seq)
    return df, fbm_seq, fbm_seq_shift


def fixed_image_precompute(image, additional_params):
    if additional_params is None:
        additional_params = {}
        
    if 'integration_intervals' in additional_params:
        Im1, Ih1, Imm, theta_net, \
        u_net, x_net, omega_net, psi_net, eta_net, eps, b, bandwidth = additional_params['integration_intervals']
    else:
        Im1, Ih1, Imm, theta_net, u_net, x_net, omega_net, psi_net, eta_net, eps, b, bandwidth  = \
            set_integration_intervals(image_radius, p_s, com_offset)
        additional_params['integration_intervals'] = \
        Im1, Ih1, Imm, theta_net, u_net, x_net, omega_net, psi_net, eta_net, eps, b, bandwidth 
    

    if 'polar_fixed' in additional_params:
        pol1 = additional_params['polar_fixed']
    else:
        maxrad = image_radius
        pol1 = polar_trfm(image, int(2 * bandwidth), int(2 * bandwidth / np.pi), maxrad)

    if 'precomputed_fbt_fixed' in additional_params:
        Fm_arr = additional_params['precomputed_fbt_fixed']
    else:
        alphas = []
        for it_m1 in range(len(Im1)):
            m1 = Im1[it_m1]
            for it_h1 in range(len(Ih1)):
                h1 = Ih1[it_h1]
                for it_mm in range(len(Imm)):
                    mm = Imm[it_mm]
                    if m1 + h1 + mm in alphas:
                        continue
                    alphas.append(m1 + h1 + mm)
        # print('Fm precompute')
        if method in ['fbm_laguerre', 'fast_fbm_laguerre'] and \
                'laguerre_functions' not in additional_params:
            additional_params['laguerre_functions'] = laguerre_functions_precompute(alphas, x_net,
                                                                                   lag_func_num, lag_scale)
        Fm_arr = image_fbt_precompute(pol1, alphas, theta_net, u_net, x_net,
                                      method, lag_func_num, lag_scale,
                                      lag_num_dots,
                                      additional_params)
        additional_params['precomputed_fbt_fixed'] = Fm_arr
    return additional_params

In [3]:
from pathlib import Path

global_path = '/home/ubuntu/Data/cryo-em samples/synthetic_data_2023'
folders = ['small_trans', 'no_trans']#['no_trans', 'small_trans', 'big_trans']
models = ['model2096', 'model6185']
names = [('_stack.mrc', '_info.csv')]#, ('_iter_stack.mrc', '_iter_info.csv')]
nums = {models[0]:2, models[1]:1}

In [4]:
results_path = Path(global_path) / 'results_0109'
os.makedirs(str(results_path), exist_ok=True)

method = 'fbm'#'fast_fbm_laguerre'
method_path = results_path / method
os.makedirs(str(method_path), exist_ok=True)


functions = {'fbm': run_fbm, 'fbm_laguerre': run_fbm_laguerre, 'fast_fbm_laguerre': run_fast_fbm_laguerre}

func = functions[method]

In [5]:
from skimage import registration as im_reg_met

image_radius = 120
pixel_sampling = 0.5
com_offset_initial = 8
lag_func_num=50
lag_scale=5
lag_num_dots = 2000
center = 128, 128



image_radius= 196
pixel_sampling = 0.8302841060319553
com_offset_initial=5.546597349024552
lag_func_num = 156
lag_scale = 10 #4.708273897046558
lag_num_dots = 5000

In [7]:
def precompute(compute_zeros=False):
    Im1, Ih1, Imm, theta_net, u_net, x_net, omega_net, psi_net, eta_net, eps, b, bandwidth = \
            set_integration_intervals(image_radius, pixel_sampling, com_offset_initial,verbose=True)
    alphas = []
    for it_m1 in tqdm(range(len(Im1))):
        m1 = Im1[it_m1]
        for it_h1 in range(len(Ih1)):
            h1 = Ih1[it_h1]
            for it_mm in range(len(Imm)):
                mm = Imm[it_mm]
                if abs(m1 + h1 + mm) in alphas:
                    continue
                alphas.append(abs(m1 + h1 + mm))

    params = {}
    if compute_zeros:
        if os.path.exists(f'cryo_laguerre_zeros_{len(alphas)}_{lag_func_num}.pkl'):
            with open(f'cryo_laguerre_zeros_{len(alphas)}_{lag_func_num}.pkl', 'rb') as file:
                params['lag_zeros'] = pickle.load(file)  
        else:
            params['lag_zeros'] = laguerre_zeros_precompute(alphas, lag_func_num+1, abort_after=0.01)
            with open(f'cryo_laguerre_zeros_{len(alphas)}_{lag_func_num}.pkl', 'wb') as file:
                pickle.dump(params['lag_zeros'], file)
    params['integration_intervals'] = [Im1, Ih1, Imm, theta_net, u_net, x_net, omega_net, 
                                   psi_net, eta_net, eps, b, bandwidth]

    laguerre_functions = laguerre_functions_precompute(alphas, x_net, lag_func_num, lag_scale)
    params['laguerre_functions'] = laguerre_functions

    c1_coefs = np.zeros((len(Im1), len(x_net)))
    for it_m1 in range(len(Im1)):
        m1 = Im1[it_m1]
        c1 = sp.special.jv(m1, b * x_net) * x_net
        c1_coefs[it_m1, :] = c1
    params['precomputed_c1_coefs'] = c1_coefs

    c2_coefs = np.zeros((len(Ih1), len(x_net)))
    for it_m1 in range(len(Im1)):
        for it_h1 in range(len(Ih1)):
            h1 = Ih1[it_h1]
            if it_m1 == 0:
                c2 = sp.special.jv(h1, b * x_net)
                c2_coefs[it_h1, :] = c2
    params['precomputed_c2_coefs'] = c2_coefs
    
    ih1, imm = np.meshgrid(Imm, Ih1)
    coef = 2 * np.pi * np.exp(1j * (ih1 + imm) * eps)
    params['precomputed_coef_exp'] = coef
    
#     coef_exp = np.zeros((len(Ih1), len(Imm)))
#     c2_coef = np.zeros((len(Ih1), len(x_net)))
#     for it_m1 in range(len(Im1)):
#         c1 = c1_coefs[it_m1, :]
#         m1 = Im1[it_m1]
#         for it_h1 in range(len(Ih1)):
#             h1 = Ih1[it_h1]
#             c2_coef[it_h1, :] = c2_coefs[it_h1, :] * c1
#             for it_mm in range(len(Imm)):
#                 mm = Imm[it_mm]
#                 coef_exp[it_h1, it_mm] = 2 * np.pi * np.exp(1j * (h1 + mm) * eps)
                
#     params['precomputed_c2'] = c2_coef
#     params['precomputed_coef_exp'] = coef_exp
    
    return params

In [9]:
params = precompute()
for folder in folders:
    for model in models:
        for path in glob(str(Path(global_path) / folder / model / f'{nums[model]}_stack.mrc')):
            print(path)
#             with mrcfile.open(path) as mrc:
#                 seq = mrc.data[:5]

#             params = precompute()

            method_path = results_path / method
            os.makedirs(str(method_path), exist_ok=True)

            opath = method_path / folder / model
            os.makedirs(opath, exist_ok=True)
            opath = method_path / folder / model / str(path).split('/')[-1]
#             if not os.path.exists(opath):
#                 df, fbm_seq, fbm_seq_shift = func(seq, params)
#                 save_arr_mrc(opath, fbm_seq)
#                 df.to_csv(str(opath).replace('stack.mrc', 'info.csv'))

        for snr_value in [0.1, 0.5, 1., 2.]:
            for data_name in glob(str(Path(global_path) / folder / model / 
                                      f'{snr_value}/{nums[model]}_stack.mrc')):
                with mrcfile.open(data_name) as mrc:
                    seq = mrc.data
                df, fbm_seq, fbm_seq_shift = func(seq, params)
                print(df.head(5))
#                 opath = method_path / folder / model / str(snr_value)
#                 os.makedirs(str(opath), exist_ok=True)
#                 opath = method_path / folder / model / str(snr_value) / data_name.split('/')[-1]
#                 save_arr_mrc(opath, fbm_seq)
#                 df.to_csv(str(opath).replace('stack.mrc', 'info.csv'))


/home/ubuntu/Data/cryo-em samples/synthetic_data_2023/small_trans/model2096/2_stack.mrc


  0%|                                                     | 0/4 [00:00<?, ?it/s]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
 50%|██████████████████████▌                      | 2/4 [00:11<00:11,  5.91s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
 75%|█████████████████████████████████▊           | 3/4 [00:17<00:05,  5.89s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/

          x         y         ang       ksi  eta_prime  omega_prime  dft_x  \
0  0.214132  1.122518   64.829244  1.382301   4.712389     1.033309  -0.27   
1 -0.010013 -0.012103   36.182029  4.021239   0.785398     0.533321   1.71   
2 -0.024270 -0.074695  158.410146  4.398230   1.178097     2.666604   1.13   
3 -1.347206  0.345904  198.516247  2.890265   6.283185     3.366587   1.89   

   dft_y  
0   3.87  
1  -0.90  
2  -1.68  
3   2.46  


In [10]:
params = precompute()

for folder in folders:
    for model in models:
        for path in glob(str(Path(global_path) / folder / model / f'{nums[model]}_stack.mrc')):
            print(path)
            with mrcfile.open(path) as mrc:
                seq = mrc.data

            method_path = results_path / method
            os.makedirs(str(method_path), exist_ok=True)

            opath = method_path / folder / model
            os.makedirs(opath, exist_ok=True)
            opath = method_path / folder / model / str(path).split('/')[-1]
#             if not os.path.exists(opath):
#                 df, fbm_seq, fbm_seq_shift = func(seq, params)
#                 save_arr_mrc(opath, fbm_seq)
#                 df.to_csv(str(opath).replace('stack.mrc', 'info.csv'))

        for snr_value in [1., 0.1, 0.5, 2.]:
            for data_name in glob(str(Path(global_path) / folder / model / 
                                      f'{snr_value}/{nums[model]}_stack.mrc')):
                with mrcfile.open(data_name) as mrc:
                    seq = mrc.data
                df, fbm_seq, fbm_seq_shift = func(seq, params)
                opath = method_path / folder / model / str(snr_value)
                os.makedirs(str(opath), exist_ok=True)
                opath = method_path / folder / model / str(snr_value) / data_name.split('/')[-1]
                save_arr_mrc(opath, fbm_seq)
                df.to_csv(str(opath).replace('stack.mrc', 'info.csv'))


/home/ubuntu/Data/cryo-em samples/synthetic_data_2023/small_trans/model2096/2_stack.mrc
120
s_ang = 753.9822368615503
s_rad = 239.99999999999997
bandwidth = 376.99111843077515
len(Im1) 51 len(Ih1) 17 len(Imm) 378


100%|███████████████████████████████████████████| 51/51 [00:00<00:00, 59.54it/s]
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
  2%|▉                                           | 2/99 [00:11<09:32,  5.90s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered

 44%|███████████████████                        | 44/99 [04:20<05:25,  5.91s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
 45%|███████████████████▌                       | 45/99 [04:26<05:21,  5.95s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
 46%|███████████████████▉                       | 46/99 [04:32<05:13,  5.92s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/

 67%|████████████████████████████▋              | 66/99 [06:34<03:25,  6.22s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
 68%|█████████████████████████████              | 67/99 [06:40<03:16,  6.13s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
 69%|█████████████████████████████▌             | 68/99 [06:46<03:08,  6.10s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/

 89%|██████████████████████████████████████▏    | 88/99 [08:48<01:08,  6.19s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
 90%|██████████████████████████████████████▋    | 89/99 [08:54<01:01,  6.14s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
 91%|███████████████████████████████████████    | 90/99 [09:00<00:54,  6.07s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/

...

 91%|█████████████████████████████████▋   | 90/99 [4:59:20<3:07:35, 1250.62s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
 92%|██████████████████████████████████▉   | 91/99 [4:59:33<1:57:12, 879.10s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
 93%|███████████████████████████████████▎  | 92/99 [4:59:45<1:12:13, 619.01s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/

 14%|██████                                     | 14/99 [02:47<16:50, 11.88s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
 15%|██████▌                                    | 15/99 [02:59<16:39, 11.90s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
 16%|██████▉                                    | 16/99 [03:11<16:21, 11.83s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/

 36%|███████████████▋                           | 36/99 [07:08<12:21, 11.77s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
 37%|████████████████                           | 37/99 [07:20<12:07, 11.73s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
 38%|████████████████▌                          | 38/99 [07:31<11:55, 11.73s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/

 59%|█████████████████████████▏                 | 58/99 [11:28<08:06, 11.86s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
 60%|█████████████████████████▋                 | 59/99 [11:40<07:52, 11.82s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
 61%|██████████████████████████                 | 60/99 [11:52<07:41, 11.84s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/

 81%|██████████████████████████████████▋        | 80/99 [15:50<03:47, 11.95s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
 82%|███████████████████████████████████▏       | 81/99 [16:01<03:35, 11.95s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:101: RuntimeWarning: invalid value encountered in multiply
  value = norm * exp * mul_alpha
 83%|███████████████████████████████████▌       | 82/99 [16:13<03:22, 11.93s/it]/home/ubuntu/projects/Fast-Bessel-Matching/laguerre.py:98: RuntimeWarning: overflow encountered in power
  mul_alpha = (a * x) ** (abs(alpha) / 2.)
/home/ubuntu/projects/

In [6]:
sys.path.append('./compute results/')
from registration_functions import precompute_w_params, run_fbm, run_fbm_laguerre, run_fast_fbm_laguerre

In [7]:
input_values = {'image_radius': image_radius,
                'pixel_sampling': pixel_sampling,
                'com_offset_initial': com_offset_initial}

if 'laguerre' in method:
    input_values['lag_func_num'] = lag_func_num
    input_values['lag_scale'] = lag_scale
    input_values['lag_num_dots'] = lag_num_dots
    

In [8]:
params = precompute_w_params(image_radius=image_radius,
                            pixel_sampling=pixel_sampling, com_offset_initial=com_offset_initial,
                             lag_func_num=lag_func_num, lag_scale=lag_scale, 
                             lag_num_dots=lag_num_dots, compute_zeros=True)

for folder in folders:
    for model in models:
        for path in glob(str(Path(global_path) / folder / model / f'{nums[model]}_stack.mrc')):
            print(path)
            with mrcfile.open(path) as mrc:
                seq = mrc.data

            method_path = results_path / method
            os.makedirs(str(method_path), exist_ok=True)

            opath = method_path / folder / model
            os.makedirs(opath, exist_ok=True)
            opath = method_path / folder / model / str(path).split('/')[-1]
#             if not os.path.exists(opath):
#                 df, fbm_seq, fbm_seq_shift = func(seq, params)
#                 save_arr_mrc(opath, fbm_seq)
#                 df.to_csv(str(opath).replace('stack.mrc', 'info.csv'))

        for snr_value in [1., 0.1, 0.5, 2.]:
            for data_name in glob(str(Path(global_path) / folder / model / 
                                      f'{snr_value}/{nums[model]}_stack.mrc')):
                with mrcfile.open(data_name) as mrc:
                    seq = mrc.data
                df, fbm_seq, fbm_seq_shift = func(seq=seq, func_parameters=params, **input_values)
                opath = method_path / folder / model / str(snr_value)
                os.makedirs(str(opath), exist_ok=True)
                opath = method_path / folder / model / str(snr_value) / data_name.split('/')[-1]
                save_arr_mrc(opath, fbm_seq)
                df.to_csv(str(opath).replace('stack.mrc', 'info.csv'))


196
s_ang = 741.6162198339141
s_rad = 236.0637745273863
bandwidth = 370.80810991695705
len(Im1) 21 len(Ih1) 7 len(Imm) 372


100%|██████████████████████████████████████████| 21/21 [00:00<00:00, 193.23it/s]


/home/ubuntu/Data/cryo-em samples/synthetic_data_2023/small_trans/model2096/2_stack.mrc


100%|███████████████████████████████████████████| 99/99 [39:12<00:00, 23.76s/it]


/home/ubuntu/Data/cryo-em samples/synthetic_data_2023/small_trans/model6185/1_stack.mrc


100%|███████████████████████████████████████████| 99/99 [39:11<00:00, 23.75s/it]


/home/ubuntu/Data/cryo-em samples/synthetic_data_2023/no_trans/model2096/2_stack.mrc


100%|███████████████████████████████████████████| 99/99 [54:30<00:00, 33.03s/it]


/home/ubuntu/Data/cryo-em samples/synthetic_data_2023/no_trans/model6185/1_stack.mrc


100%|███████████████████████████████████████████| 99/99 [45:47<00:00, 27.75s/it]
